In [25]:
from __future__ import print_function, division
import pandas as pd
from collections import Counter
import unicodecsv
import numpy as np
import random
import sys
import os
import copy
import csv
from math import log
from sklearn import preprocessing
pd.set_option('display.max_rows', None)

from datetime import datetime
import time

#define the maximum length of events
def find_max_list(lst):
    list_len = [len(i) for i in lst]
    return max(list_len)


filename = 'event_data_aggregated_alex2'
#read in original data
df = pd.read_csv('data/' + filename +'.csv')

#only keep the relevant columns
df_orig = df[['CaseID','ActivityID','CompleteTimestamp']]

#labelencode the activity labels
le_act = preprocessing.LabelEncoder()

#labelencode the case IDs
le_case = preprocessing.LabelEncoder()

# Use .loc to avoid the SettingWithCopyWarning
df_orig.loc[:, 'ActivityID'] = le_act.fit_transform(df_orig['ActivityID']) + 1
df_orig.loc[:, 'CaseID'] = le_case.fit_transform(df_orig['CaseID']) + 1

df_orig['CaseID'].nunique()

#sort values by CaseID
df_new = df_orig.sort_values(['CaseID','CompleteTimestamp'])

#save and load again
df_new.to_csv('data/' + filename + '_reworked.csv', index=False)
df_new = pd.read_csv('data/' + filename + '_reworked.csv')
print('the shape of the data',df_new.shape)

the shape of the data (3723, 3)


In [10]:
########################################################################################
#this basically creates lists of lists with all the relevant information
########################################################################################

eventlog = filename + '_reworked'
csvfile = open('data/%s' % eventlog+'.csv', 'r')
datareader = csv.reader(csvfile, delimiter=',', quotechar='|')

#helper variables
# Load data from CSV file
lineseq, timeseqs, timeseqs2 = [], [], []
lastcase, line, firstLine = '', '', True
times, times2, numlines = [], [], 0
casestarttime, lasteventtime = None, None
next(datareader, None)  # skip the headers
for row in datareader: #the columns are "CaseID,ActivityID,CompleteTimestamp"
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S") #creates a datetime object from row[2]
    if row[0]!=lastcase:  #'lastcase' is to save the last executed case for the loop
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:
            lines.append('!')
            lineseq.append(lines)
            timeseqs.append(times)
            timeseqs2.append(times2)

        lines = []
        times = []
        times2 = []
        times3 = []
        times4 = []
        numlines+=1
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    midnight = datetime.fromtimestamp(time.mktime(t)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(t))-midnight
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds     #multiply with 60*60*24 = 86400 to go from days to seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds    #the .seconds method gives the time in seconds
    timediff3 = timesincemidnight.seconds #this leaves only time event occured after midnight
    timediff4 = datetime.fromtimestamp(time.mktime(t)).weekday() #day of the week
    lines.append(str(row[1])) #add the activity label to the line list
    times.append(timediff)
    times2.append(timediff2)
    times3.append(timediff3)
    times4.append(timediff4)
    lasteventtime = t
    index = index+1
    firstLine = False #after the each case, we set the FirstLine to False; this is because we want to save the lines and times lists to the lineseq and timeseq list for each seperate case

# add last case
lines.append('!')
lineseq.append(lines)
timeseqs.append(times)
timeseqs2.append(times2)
timeseqs3.append(times3)
timeseqs4.append(times4)
numlines+=1

divisor = np.mean([item for sublist in timeseqs for item in sublist]) #average time between events
print('divisor: {}'.format(divisor))
divisor2 = np.mean([item for sublist in timeseqs2 for item in sublist]) #average time between current and first events
print('divisor2: {}'.format(divisor2))

#########################################################################################################
# separate data into 3 parts
elems_per_fold = int(round(numlines/3))
fold1 = lineseq[:elems_per_fold]
fold1_t = timeseqs[:elems_per_fold]
fold1_t2 = timeseqs2[:elems_per_fold]
fold1_t3 = timeseqs3[:elems_per_fold]
fold1_t4 = timeseqs4[:elems_per_fold]
import csv
with open('code/output_files/folds/fold1.csv', 'w', newline='') as csvfile:
    datawriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in zip(fold1, fold1_t):
        datawriter.writerow([f"{s}#{t}" for s, t in zip(row, timeseq)])


fold2 = lineseq[elems_per_fold:2*elems_per_fold]
fold2_t = timeseqs[elems_per_fold:2*elems_per_fold]
fold2_t2 = timeseqs2[elems_per_fold:2*elems_per_fold]
fold2_t3 = timeseqs3[elems_per_fold:2*elems_per_fold]
fold2_t4 = timeseqs4[elems_per_fold:2*elems_per_fold]
with open('code/output_files/folds/fold2.csv', 'w', newline='') as csvfile:
    datawriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in zip(fold2, fold2_t):
        datawriter.writerow([f"{s}#{t}" for s, t in zip(row, timeseq)])

fold3 = lineseq[2*elems_per_fold:]
fold3_t = timeseqs[2*elems_per_fold:]
fold3_t2 = timeseqs2[2*elems_per_fold:]
fold3_t3 = timeseqs3[2*elems_per_fold:]
fold3_t4 = timeseqs4[2*elems_per_fold:]
with open('code/output_files/folds/fold3.csv','w', newline='') as csvfile:
    datawriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in zip(fold3, fold3_t):
        datawriter.writerow([f"{s}#{t}" for s, t in zip(row, timeseq)])

divisor: 21465.032500671503
divisor2: 242783.27692720923


In [11]:
lines = fold1 + fold2
lines_t = fold1_t + fold2_t
lines_t2 = fold1_t2 + fold2_t2
lines_t3 = fold1_t3 + fold2_t3
lines_t4 = fold1_t4 + fold2_t4

step = 1
sentences = []
softness = 0
next_chars = []

In [12]:
#helper variables
step = 1
sentences = []
softness = 0
next_chars = []
sentences_t = []
sentences_t2 = []
sentences_t3 = []
sentences_t4 = []
next_chars_t = []
next_chars_t2 = []
next_chars_t3 = []
next_chars_t4 = []
maxlen = find_max_list(lines) #find maximum line size

for line, line_t, line_t2, line_t3, line_t4 in zip(lines, lines_t, lines_t2, lines_t3, lines_t4):
    for i in range(0, len(line), step):
        if i==0:
            continue
        #we take all the prefix traces of the traces
        #we add iteratively, first symbol of the line, then two first, three...
        sentences.append(line[0:i])
        sentences_t.append(line_t[0:i])
        sentences_t2.append(line_t2[0:i])
        sentences_t3.append(line_t3[0:i])
        sentences_t4.append(line_t4[0:i])
        next_chars.append(line[i])
        if i==len(line)-1: # special case to deal time of end character
            next_chars_t.append(0)
            next_chars_t2.append(0)
            next_chars_t3.append(0)
            next_chars_t4.append(0)
        else:
            next_chars_t.append(line_t[i])
            next_chars_t2.append(line_t2[i])
            next_chars_t3.append(line_t3[i])
            next_chars_t4.append(line_t4[i])
activities = set([item for sublist in lineseq for item in sublist])
activities.remove('!')
target_activities = copy.copy(activities)
print('there are now:', len(sentences), 'prefix sequences')
print('there are:', len(activities), 'different activities')

there are now: 2445 prefix sequences
there are: 15 different activities


In [13]:
print('total activities: {}, target activities: {}'.format(len(activities), len(target_activities)))
act_indices = dict((c, i) for i, c in enumerate(activities))
indices_act = dict((i, c) for i, c in enumerate(activities))
target_activities_indices = dict((c, i) for i, c in enumerate(target_activities))
target_indices_activities = dict((i, c) for i, c in enumerate(target_activities))
print(indices_act)

print('Vectorization...')
num_features = len(activities)+5
print('num features: {}'.format(num_features))
X = np.zeros((len(sentences), maxlen, num_features), dtype=np.float32)
y_a = np.zeros((len(sentences), len(target_activities)), dtype=np.float32)
y_t = np.zeros((len(sentences)), dtype=np.float32)

total activities: 15, target activities: 15
{0: '2', 1: '6', 2: '13', 3: '12', 4: '1', 5: '15', 6: '10', 7: '9', 8: '4', 9: '14', 10: '7', 11: '5', 12: '8', 13: '3', 14: '11'}
Vectorization...
num features: 20


In [14]:
print(maxlen)

45


In [15]:
for i, sentence in enumerate(sentences):
    leftpad = maxlen-len(sentence)
    next_t = next_chars_t[i]
    sentence_t = sentences_t[i]
    sentence_t2 = sentences_t2[i]
    sentence_t3 = sentences_t3[i]
    sentence_t4 = sentences_t4[i]
    for t, char in enumerate(sentence):
        multiset_abstraction = Counter(sentence[:t+1])
        for c in activities:
            if c==char: #this will encode present events to the right places
                X[i, t+leftpad, act_indices[c]] = 1
        X[i, t+leftpad, len(activities)] = t+1
        X[i, t+leftpad, len(activities)+1] = sentence_t[t]/divisor
        X[i, t+leftpad, len(activities)+2] = sentence_t2[t]/divisor2
        X[i, t+leftpad, len(activities)+3] = sentence_t3[t]/86400
        X[i, t+leftpad, len(activities)+4] = sentence_t4[t]/7
    for c in target_activities:
        if c==next_chars[i]:
            y_a[i, target_activities_indices[c]] = 1-softness
        else:
            y_a[i, target_activities_indices[c]] = softness/(len(target_activities)-1)
    y_t[i] = next_t/divisor

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Dense, Input, BatchNormalization
from tensorflow.keras.utils import get_file
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# build the model: 
print('Build model...')
main_input = Input(shape=(maxlen, num_features), name='main_input')
# train a 2-layer LSTM with one shared layer
l1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=True, dropout=0.2)(main_input) # the shared layer
b1 = BatchNormalization()(l1)
l2_1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(b1) # the layer specialized in activity prediction
b2_1 = BatchNormalization()(l2_1)

act_output = Dense(len(target_activities), activation='softmax', kernel_initializer='glorot_uniform', name='act_output')(b2_1)

model = Model(inputs=[main_input], outputs=act_output)

opt = Nadam(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipvalue=3)

model.compile(loss={'act_output':'categorical_crossentropy'}, optimizer=opt)
early_stopping = EarlyStopping(monitor='val_loss', patience=42)
model_checkpoint = ModelCheckpoint('output_files/models/model_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

model.fit(X, {'act_output':y_a}, validation_split=0.2, verbose=2, callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=maxlen, epochs=100)


Build model...

Epoch 1/100

44/44 - 8s - loss: 2.5199 - val_loss: 2.3943 - lr: 0.0020 - 8s/epoch - 181ms/step
Epoch 2/100


c:\Users\u0138175\AppData\Local\anaconda3\envs\IOTPPM\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


44/44 - 2s - loss: 1.9474 - val_loss: 2.2420 - lr: 0.0020 - 2s/epoch - 40ms/step
Epoch 3/100
44/44 - 2s - loss: 1.6512 - val_loss: 2.0437 - lr: 0.0020 - 2s/epoch - 39ms/step
Epoch 4/100
44/44 - 2s - loss: 1.4878 - val_loss: 1.9249 - lr: 0.0020 - 2s/epoch - 39ms/step
Epoch 5/100
44/44 - 2s - loss: 1.3416 - val_loss: 1.8119 - lr: 0.0020 - 2s/epoch - 41ms/step
Epoch 6/100
44/44 - 2s - loss: 1.2424 - val_loss: 1.6081 - lr: 0.0020 - 2s/epoch - 41ms/step
Epoch 7/100
44/44 - 2s - loss: 1.1953 - val_loss: 1.5427 - lr: 0.0020 - 2s/epoch - 42ms/step
Epoch 8/100
44/44 - 2s - loss: 1.1596 - val_loss: 1.6065 - lr: 0.0020 - 2s/epoch - 40ms/step
Epoch 9/100
44/44 - 2s - loss: 1.1846 - val_loss: 1.3446 - lr: 0.0020 - 2s/epoch - 40ms/step
Epoch 10/100
44/44 - 2s - loss: 1.1629 - val_loss: 1.3652 - lr: 0.0020 - 2s/epoch - 39ms/step
Epoch 11/100
44/44 - 2s - loss: 1.1273 - val_loss: 1.4243 - lr: 0.0020 - 2s/epoch - 39ms/step
Epoch 12/100
44/44 - 2s - loss: 1.1715 - val_loss: 1.0627 - lr: 0.0020 - 2s/epoc